In [2]:
import numpy as np
import json
from itertools import product

In [3]:
data = json.load(open("output.json"))

In [4]:
current_data = data["active"]["current"]
current_matrix = np.array(current_data["JM"])

transport_data = data["active"]["TransportXS"]
transportXS_matrix = np.array(transport_data["transportFluxLimited"])
flux_matrix = np.array(transport_data["fluxG"])

# TODO: properly extract spacing
spacing = {"x":4.0, "y": 4.0, "z": 4.0}

In [5]:
max_x = len(current_data["XBounds"])
max_y = len(current_data["YBounds"])
max_z = len(current_data["ZBounds"])
max_energy_group = len(transport_data["EnergyBounds"][0])

In [6]:
print(current_matrix.shape)

print(flux_matrix.shape)
print(transportXS_matrix.shape)

(125, 3, 7, 2)
(5, 5, 5, 7, 2)
(5, 5, 5, 7, 2)


In [7]:
def coord_to_multimap_index(x, y, z):
    return z * max_x * max_y + y * max_x + x

def get_x_current_on_cell_right_boundary(x, y, z, energy_group):
    # coord, direction, energy, mean/ std
    return current_matrix[coord_to_multimap_index(x, y, z), 0, energy_group, 0]

def get_flux_at_coordinate(x, y, z, energy_group):
    return flux_matrix[z, y, x, energy_group, 0]

def get_transportXS_at_coordinate(x, y, z, energy_group):
    return transportXS_matrix[z, y, x, energy_group, 0]

In [8]:
diffusion_coefficient = np.zeros((max_x, max_y, max_z, max_energy_group))

for x, y, z, energy in product(range(max_x), range(max_y), range(max_z), range(max_energy_group)):
    transport_XS = get_transportXS_at_coordinate(x, y, z, energy)
    if transport_XS == 0:
        continue
    diffusion_coefficient[x, y, z, energy] = 1.0 / transport_XS

In [17]:
# linear cell-to-x coupling in x direction
current_contribution_due_to_x_linear_coupling = np.zeros((max_x, max_y, max_z, max_energy_group))

for x, y, z, energy in product(range(max_x), range(max_y), range(max_z), range(max_energy_group)):
    if x+1 >= max_x:
        # TODO: cell to boundary coupling
        continue

    # Cell-to-cell coupling
    D = diffusion_coefficient[x, y, z, energy]
    D_plus_1 = diffusion_coefficient[x+1, y, z, energy]

    if D == 0 and D_plus_1 == 0:
        continue  # What should happen here?

    coupling = -(2 * D_plus_1 * D)/(spacing["y"]*spacing["z"] * (D_plus_1 + D))

    flux = get_flux_at_coordinate(x, y, z, energy)
    flux_plus_1 = get_flux_at_coordinate(x+1, y, z, energy)

    # TODO: what about the negative direction
    current_contribution_due_to_x_linear_coupling[x, y, z, energy] = coupling * (flux_plus_1 - flux)

In [18]:
non_linear_parameter = np.zeros((max_x, max_y, max_z, max_energy_group))

for x, y, z, energy in product(range(max_x), range(max_y), range(max_z), range(max_energy_group)):
    if x+1 >= max_x:
        # TODO: cell to boundary coupling
        continue

    # Cell-to-cell coupling
    j_linear = current_contribution_due_to_x_linear_coupling[x, y, z, energy]
    flux = get_flux_at_coordinate(x, y, z, energy)
    if flux == 0:
        continue  # TODO: what to do here?

    j_x = get_x_current_on_cell_right_boundary(x, y, z, energy)
    non_linear_parameter[x, y, z, energy] = (j_x - j_linear) / flux

In [21]:
non_linear_parameter[:, :, 3, 5]

array([[-0.01370159, -0.00620131, -0.02612   ,  0.01538584, -0.01269533],
       [ 0.06918026, -0.0223024 , -0.00433593,  0.01521   ,  0.02163143],
       [-0.04037611,  0.00903076,  0.02588336,  0.00283336, -0.03865517],
       [-0.00366029,  0.0161489 ,  0.01638859, -0.02650781,  0.01691476],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])